# Assignment 4: K-Means and Hierarchical Clustering
Group K:
Shehab Hassani 06071687
Fares Gharbi 06052076
Riddihma
Leni

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

## Section 1: Data Loading and Z-Score Normalization

Load the `customers.csv` dataset and apply Z-score normalization on the numerical features: **Age**, **Income**, and **Score**.

In [ ]:
# Load the dataset
df = pd.read_csv('customers.csv')
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head(10)

In [ ]:
# Z-score normalisation on numerical features: Age, Income, Score
numerical_features = ['Age', 'Income', 'Score']
scaler = StandardScaler()
df_normalised = df.copy()
df_normalised[numerical_features] = scaler.fit_transform(df[numerical_features])

print("Original data (first 5 rows):")
print(df[numerical_features].head())
print("\nNormalised data (first 5 rows):")
print(df_normalised[numerical_features].head())
print(f"\nNormalised means (should be ~0): {df_normalised[numerical_features].mean().values}")
print(f"Normalised stds  (should be ~1): {df_normalised[numerical_features].std().values}")

## Section 2: K-Means with k = 2..10 and Best k Selection

Run K-means for k = 2, 3, ..., 10 using the normalised features. Use the **Elbow Method** (inertia / WCSS) and **Silhouette Score** to determine the best k.

In [ ]:
X_normalised = df_normalised[numerical_features].values

# Run K-means for k = 2..10
k_range = range(2, 11)
inertias = []
silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = kmeans.fit_predict(X_normalised)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_normalised, labels))

# Display results
results_df = pd.DataFrame({
    'k': list(k_range),
    'Inertia (WCSS)': inertias,
    'Silhouette Score': silhouette_scores
})
print(results_df.to_string(index=False))

In [ ]:
# Plot Elbow Method and Silhouette Score
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elbow Method
axes[0].plot(list(k_range), inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0].set_ylabel('Inertia (WCSS)', fontsize=12)
axes[0].set_title('Elbow Method', fontsize=14)
axes[0].set_xticks(list(k_range))
axes[0].grid(True, alpha=0.3)

# Silhouette Score
axes[1].plot(list(k_range), silhouette_scores, 'ro-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score', fontsize=14)
axes[1].set_xticks(list(k_range))
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Determine the best k using the silhouette score
best_k = list(k_range)[np.argmax(silhouette_scores)]
print(f"\nBest k (highest silhouette score): {best_k}")
print(f"Silhouette score at k={best_k}: {max(silhouette_scores):.4f}")
print(
    f"\nThe elbow method shows a noticeable bend, "
    f"while the silhouette score is maximised at k={best_k}. "
    f"We select k={best_k} as the optimal number of clusters."
)

## Section 3: 3D Cluster Visualisation with Best k

Cluster the samples using K-means with the best k. Plot the clusters and centroids in 3D using the **denormalised** (original) axes.

In [ ]:
# Fit K-means with the best k
kmeans_best = KMeans(n_clusters=best_k, n_init=10, random_state=42)
cluster_labels = kmeans_best.fit_predict(X_normalised)

# Denormalise the centroids back to original scale
centroids_normalised = kmeans_best.cluster_centers_
centroids_original = scaler.inverse_transform(centroids_normalised)

# Add cluster labels to the dataframe
df['Cluster'] = cluster_labels

# Print centroid information
centroid_df = pd.DataFrame(
    centroids_original,
    columns=numerical_features,
    index=[f'Cluster {i}' for i in range(best_k)]
)
print(f"K-Means with k={best_k} — Cluster Centroids (original scale):\n")
print(centroid_df.round(2))
print(f"\nCluster sizes:")
print(df['Cluster'].value_counts().sort_index())

In [ ]:
# 3D scatter plot with denormalised axes
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

# Use a colormap for distinct cluster colours
colours = plt.cm.tab10(np.linspace(0, 1, best_k))

for i in range(best_k):
    mask = cluster_labels == i
    ax.scatter(
        df.loc[mask, 'Age'],
        df.loc[mask, 'Income'],
        df.loc[mask, 'Score'],
        c=[colours[i]],
        label=f'Cluster {i}',
        alpha=0.6,
        s=40
    )

# Plot centroids
ax.scatter(
    centroids_original[:, 0],
    centroids_original[:, 1],
    centroids_original[:, 2],
    c='black',
    marker='X',
    s=200,
    edgecolors='white',
    linewidths=1.5,
    label='Centroids',
    zorder=5
)

ax.set_xlabel('Age', fontsize=12)
ax.set_ylabel('Income (k$)', fontsize=12)
ax.set_zlabel('Spending Score', fontsize=12)
ax.set_title(f'Customer Clusters (k={best_k}) — 3D View', fontsize=14)
ax.legend(fontsize=10, loc='upper left')
plt.tight_layout()
plt.show()

### Interpretation of Customer Segments

The K-means clustering reveals distinct customer segments based on their Age, Annual Income, and Spending Score. Typical segments that emerge include:

- **High Income, High Spending Score**: Premium customers who earn well and spend generously — ideal targets for loyalty programmes and premium products.
- **High Income, Low Spending Score**: Wealthy but cautious spenders — potential targets for targeted marketing to increase engagement.
- **Low Income, High Spending Score**: Budget-conscious but enthusiastic shoppers — could benefit from discount programmes.
- **Low Income, Low Spending Score**: Low-engagement customers — may require different strategies to attract.
- **Average across all features**: The "mainstream" customer segment.

The exact composition depends on the data; examine the centroid table above for precise segment characteristics.

## Section 4: Pairwise Feature K-Means

Create three datasets with two out of three normalised features: **(Age, Income)**, **(Age, Score)**, **(Income, Score)**. Find the best k for each pair.

In [ ]:
# Define the three feature pairs
feature_pairs = [
    ('Age', 'Income'),
    ('Age', 'Score'),
    ('Income', 'Score')
]

k_range_pairs = range(2, 11)
pair_results = {}

for pair in feature_pairs:
    pair_name = f"({pair[0]}, {pair[1]})"
    X_pair = df_normalised[list(pair)].values

    inertias_pair = []
    sil_scores_pair = []

    for k in k_range_pairs:
        km = KMeans(n_clusters=k, n_init=10, random_state=42)
        labels_pair = km.fit_predict(X_pair)
        inertias_pair.append(km.inertia_)
        sil_scores_pair.append(silhouette_score(X_pair, labels_pair))

    best_k_pair = list(k_range_pairs)[np.argmax(sil_scores_pair)]
    pair_results[pair_name] = {
        'features': pair,
        'inertias': inertias_pair,
        'silhouette_scores': sil_scores_pair,
        'best_k': best_k_pair,
        'best_silhouette': max(sil_scores_pair)
    }

    print(f"{pair_name}: Best k = {best_k_pair} "
          f"(Silhouette = {max(sil_scores_pair):.4f})")

In [ ]:
# Plot Elbow and Silhouette for each feature pair
fig, axes = plt.subplots(3, 2, figsize=(14, 14))

for idx, (pair_name, res) in enumerate(pair_results.items()):
    # Elbow Method
    axes[idx, 0].plot(
        list(k_range_pairs), res['inertias'],
        'bo-', linewidth=2, markersize=6
    )
    axes[idx, 0].axvline(
        x=res['best_k'], color='red', linestyle='--',
        alpha=0.7, label=f'Best k={res["best_k"]}'
    )
    axes[idx, 0].set_xlabel('k')
    axes[idx, 0].set_ylabel('Inertia (WCSS)')
    axes[idx, 0].set_title(f'{pair_name} — Elbow Method')
    axes[idx, 0].set_xticks(list(k_range_pairs))
    axes[idx, 0].legend()
    axes[idx, 0].grid(True, alpha=0.3)

    # Silhouette Score
    axes[idx, 1].plot(
        list(k_range_pairs), res['silhouette_scores'],
        'ro-', linewidth=2, markersize=6
    )
    axes[idx, 1].axvline(
        x=res['best_k'], color='blue', linestyle='--',
        alpha=0.7, label=f'Best k={res["best_k"]}'
    )
    axes[idx, 1].set_xlabel('k')
    axes[idx, 1].set_ylabel('Silhouette Score')
    axes[idx, 1].set_title(f'{pair_name} — Silhouette Score')
    axes[idx, 1].set_xticks(list(k_range_pairs))
    axes[idx, 1].legend()
    axes[idx, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Section 5: Gender-Based Cluster Analysis

Plot the clusters for each feature pair and distinguish data points by Gender. Investigate whether gender reveals customer sub-segments.

In [ ]:
# Cluster each feature pair with its best k and visualise with gender
gender_markers = {'Male': 'o', 'Female': 's'}
gender_colours = {'Male': 'steelblue', 'Female': 'coral'}

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, (pair_name, res) in enumerate(pair_results.items()):
    feat1, feat2 = res['features']
    best_k_pair = res['best_k']
    X_pair = df_normalised[[feat1, feat2]].values

    # Fit K-means with the best k for this pair
    km_pair = KMeans(n_clusters=best_k_pair, n_init=10, random_state=42)
    pair_labels = km_pair.fit_predict(X_pair)

    # Plot with gender differentiation
    ax = axes[idx]
    for gender in ['Male', 'Female']:
        mask = df['Gender'] == gender
        scatter = ax.scatter(
            df.loc[mask, feat1],
            df.loc[mask, feat2],
            c=pair_labels[mask],
            cmap='tab10',
            marker=gender_markers[gender],
            edgecolors='black',
            linewidths=0.5,
            alpha=0.7,
            s=50,
            label=gender,
            vmin=0,
            vmax=best_k_pair - 1
        )

    # Denormalise centroids for this pair
    pair_scaler_mean = scaler.mean_[
        [numerical_features.index(feat1),
         numerical_features.index(feat2)]
    ]
    pair_scaler_std = scaler.scale_[
        [numerical_features.index(feat1),
         numerical_features.index(feat2)]
    ]
    centroids_pair_orig = (
        km_pair.cluster_centers_ * pair_scaler_std + pair_scaler_mean
    )
    ax.scatter(
        centroids_pair_orig[:, 0],
        centroids_pair_orig[:, 1],
        c='black', marker='X', s=200,
        edgecolors='white', linewidths=1.5,
        zorder=5, label='Centroids'
    )

    ax.set_xlabel(feat1, fontsize=12)
    ax.set_ylabel(feat2, fontsize=12)
    ax.set_title(f'{pair_name} — k={best_k_pair}', fontsize=13)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Gender distribution within each cluster (per feature pair)
for pair_name, res in pair_results.items():
    feat1, feat2 = res['features']
    X_pair = df_normalised[[feat1, feat2]].values
    km_pair = KMeans(
        n_clusters=res['best_k'], n_init=10, random_state=42
    )
    pair_labels = km_pair.fit_predict(X_pair)

    print(f"\n{'='*50}")
    print(f"Feature pair: {pair_name} (k={res['best_k']})")
    print('='*50)
    cross_tab = pd.crosstab(
        pair_labels, df['Gender'],
        margins=True, margins_name='Total'
    )
    cross_tab.index.name = 'Cluster'
    print(cross_tab)

### Analysis: Gender and Sub-Segments

By overlaying Gender on the 2D cluster plots, we can observe:

1. **Gender distribution across clusters**: The cross-tabulation tables above show how Male and Female customers are distributed within each cluster. If gender is roughly uniformly distributed across clusters, it suggests that gender does not strongly define sub-segments beyond what Age, Income, and Score already capture.

2. **Sub-segments**: Looking at the scatter plots, if certain clusters show a clear dominance of one gender, this hints at gender-based sub-segments (e.g., younger females with high spending scores, or older males with moderate income).

3. **Should Gender be included in K-Means?**
   - If gender shows a roughly equal split across all clusters, including it as a binary variable would add little value and may introduce noise due to the scale mismatch between a binary variable and continuous normalised features.
   - If there is a clear gender imbalance in specific clusters, encoding Gender as a binary variable (0/1) could help K-means detect finer sub-segments. However, one must be cautious about the weight a binary variable carries relative to continuous features.
   - In general, for this dataset, Gender tends to be fairly evenly distributed across clusters, suggesting it does **not** add significant discriminative power to the K-means algorithm. The existing numerical features already capture the main customer segments effectively.

## Section 6: Hierarchical Clustering on the Noisy Dataset

Load `customers_noisy.csv`, which contains the four original features (Gender is now binary) plus four noisy features. Perform hierarchical clustering on all features and plot the dendrogram.

In [ ]:
# Load the noisy dataset
df_noisy = pd.read_csv('customers_noisy.csv')
print(f"Dataset shape: {df_noisy.shape}")
print(f"Columns: {list(df_noisy.columns)}")
df_noisy.head()

In [ ]:
# Z-score normalise all features (Gender is already binary, but we
# normalise all to ensure equal weighting in distance computation)
noisy_features = df_noisy.columns.tolist()
scaler_noisy = StandardScaler()
X_noisy_normalised = scaler_noisy.fit_transform(df_noisy[noisy_features])

print("Normalised noisy dataset (first 5 rows):")
print(pd.DataFrame(
    X_noisy_normalised,
    columns=noisy_features
).head())

In [ ]:
# Perform hierarchical (agglomerative) clustering using Ward's method
Z = linkage(X_noisy_normalised, method='ward', metric='euclidean')

# Plot the dendrogram
fig, ax = plt.subplots(figsize=(16, 8))
dendrogram(
    Z,
    truncate_mode='lastp',
    p=30,
    leaf_rotation=90,
    leaf_font_size=9,
    show_contracted=True,
    ax=ax
)
ax.set_xlabel('Sample Index (or cluster size)', fontsize=12)
ax.set_ylabel('Distance (Ward)', fontsize=12)
ax.set_title(
    'Hierarchical Clustering Dendrogram — Noisy Dataset (All Features)',
    fontsize=14
)
ax.axhline(y=50, color='r', linestyle='--', alpha=0.7,
           label='Possible cut threshold')
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# Full dendrogram (no truncation) for detailed view
fig, ax = plt.subplots(figsize=(20, 8))
dendrogram(
    Z,
    leaf_rotation=90,
    leaf_font_size=6,
    ax=ax
)
ax.set_xlabel('Sample Index', fontsize=12)
ax.set_ylabel('Distance (Ward)', fontsize=12)
ax.set_title(
    'Full Dendrogram — Noisy Dataset (All Features)',
    fontsize=14
)
plt.tight_layout()
plt.show()

# Compare: hierarchical clustering on original features only
original_cols = ['Gender', 'Age', 'Income', 'Score']
X_orig_only = scaler_noisy.fit_transform(df_noisy[original_cols])
Z_orig = linkage(X_orig_only, method='ward', metric='euclidean')

fig, ax = plt.subplots(figsize=(16, 8))
dendrogram(
    Z_orig,
    truncate_mode='lastp',
    p=30,
    leaf_rotation=90,
    leaf_font_size=9,
    show_contracted=True,
    ax=ax
)
ax.set_xlabel('Sample Index (or cluster size)', fontsize=12)
ax.set_ylabel('Distance (Ward)', fontsize=12)
ax.set_title(
    'Hierarchical Clustering Dendrogram — Original Features Only '
    '(no noisy features)',
    fontsize=14
)
plt.tight_layout()
plt.show()

### Interpretation of Hierarchical Clustering on Noisy Data

**Observations from the dendrogram:**

1. **Effect of noisy features**: The four additional noisy features (Noisy1–Noisy4) introduce random variation that obscures the true cluster structure. When clustering on all 8 features, the dendrogram tends to show a less clear hierarchical structure — the merge distances become more uniform, and it is harder to identify a natural number of clusters.

2. **Comparison with original features**: When we cluster using only the 4 original features (Gender, Age, Income, Score), the dendrogram shows more distinct cluster separations with larger gaps at key merge points, making it easier to identify a reasonable number of clusters (typically around 5, consistent with our K-means analysis).

3. **Impact of noise**: The noisy features dilute the signal from the meaningful features. In high-dimensional noisy spaces, the distances between all pairs of points become more similar (the "curse of dimensionality"), making it harder for hierarchical clustering to find well-separated groups.

4. **Practical takeaway**: Before applying clustering, it is important to perform feature selection or dimensionality reduction (e.g., PCA) to remove irrelevant or noisy features. Including noise in the distance computation degrades clustering quality and leads to less interpretable results.